In [ ]:
# import library
import pandas as pd

# 1. Membaca dataset
cbf_df = pd.read_excel('cbf_data.xlsx') 
cbf_df.head()

,NAMA PERUSAHAAN,KATEGORI,INDUSTRI,KETERANGAN,deksripsi
0,Cahaya Matahari,Micro,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...
1,Laborindo Sarana,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...
2,Telering,SME,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...
3,MCI ERP,SME,Mining,NaN,Aplikasi penjualan barang untuk melakukan penj...
4,Faito,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...


In [ ]:
# 2. Data cleaning
# Cek kolom yang terdapat null
cbf_df.isnull().sum()

# Gunakan data hanya yang description tidak null
cbf_df = cbf_df[cbf_df['deksripsi'].notnull()]


In [ ]:
cbf_df["combine"] = cbf_df["KATEGORI"]+" "+ cbf_df["INDUSTRI"]+" "+cbf_df["deksripsi"]
cbf_df.head()

,NAMA PERUSAHAAN,KATEGORI,INDUSTRI,KETERANGAN,deksripsi,combine
0,Cahaya Matahari,Micro,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...,Micro Trading Aplikasi penjualan barang untuk ...
1,Laborindo Sarana,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...,Middle Manufaktur Aplikasi penjualan barang un...
2,Telering,SME,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...,SME Trading Aplikasi penjualan barang untuk me...
3,MCI ERP,SME,Mining,NaN,Aplikasi penjualan barang untuk melakukan penj...,SME Mining Aplikasi penjualan barang untuk mel...
4,Faito,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...,Middle Manufaktur Aplikasi penjualan barang un...


In [ ]:

# 3. Text Processing
# Impport library TFIDF dan Sastrawi
! pip install sastrawi
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import re
import random

# Gunakan library Sastrawi untuk stopword bahasa indonesia
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
sastrawi = StopWordRemoverFactory()
stopworda = sastrawi.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

cbf_df['desc_clean'] = cbf_df['combine'].apply(clean_text)
cbf_df.head()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.0 MB/s 


,NAMA PERUSAHAAN,KATEGORI,INDUSTRI,KETERANGAN,deksripsi,combine,desc_clean
0,Cahaya Matahari,Micro,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...,Micro Trading Aplikasi penjualan barang untuk ...,micro trading aplikasi jual barang laku jual l...
1,Laborindo Sarana,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...,Middle Manufaktur Aplikasi penjualan barang un...,middle manufaktur aplikasi jual barang laku ju...
2,Telering,SME,Trading,NaN,Aplikasi penjualan barang untuk melakukan penj...,SME Trading Aplikasi penjualan barang untuk me...,sme trading aplikasi jual barang laku jual leb...
3,MCI ERP,SME,Mining,NaN,Aplikasi penjualan barang untuk melakukan penj...,SME Mining Aplikasi penjualan barang untuk mel...,sme mining aplikasi jual barang laku jual lebi...
4,Faito,Middle,Manufaktur,NaN,Aplikasi penjualan barang untuk melakukan penj...,Middle Manufaktur Aplikasi penjualan barang un...,middle manufaktur aplikasi jual barang laku ju...


In [ ]:

# 4. Hitung TF-IDF dan Cosine similarity
cbf_df.set_index('NAMA PERUSAHAAN', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0,)
tfidf_matrix = tf.fit_transform(cbf_df['desc_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim




array([[1.        , 0.86332849, 0.93935406, ..., 0.01352037, 0.00714117,
        0.00809804],
       [0.86332849, 1.        , 0.87574772, ..., 0.01887331, 0.01238876,
        0.        ],
       [0.93935406, 0.87574772, 1.        , ..., 0.01371486, 0.0072439 ,
        0.        ],
       ...,
       [0.01352037, 0.01887331, 0.01371486, ..., 1.        , 0.03934865,
        0.        ],
       [0.00714117, 0.01238876, 0.0072439 , ..., 0.03934865, 1.        ,
        0.00132343],
       [0.00809804, 0.        , 0.        , ..., 0.        , 0.00132343,
        1.        ]])

In [ ]:
indices = pd.Series(cbf_df.index)
indices[:15]

0                        Cahaya Matahari
1                       Laborindo Sarana
2                               Telering
3                                MCI ERP
4                                  Faito
5                                Transfb
6                          Nature Market
7             PT Prima Inovasindo Sukses
8           Bahtera Indo Amplas Gemilang
9            PT Citra Perkakas Indonesia
10    PT Liza Christina Garment Industry
11                      Kementrian Agama
12                     PT Nobel Riggindo
13       PT Family Indo Pratama Industry
14                      PT Linesia Adofa
Name: NAMA PERUSAHAAN, dtype: object

In [ ]:
# 5. Modelling recommender system
def recommendations(name, top = 10):
    
    recommended = []
    
    idx = indices[indices == name].index[0]
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    top = top + 1
    top_indexes = list(score_series.iloc[0:top].index)
    
    for i in top_indexes:
        recommended.append(list(cbf_df.index)[i]+" - "+str(score_series[i]))
        
    return recommended

# Contoh menampilkan rekomendasi perusahaan
recommendations('Cahaya Matahari', 15)

['Cahaya Matahari - 1.0000000000000004',
 'Nature Market - 1.0000000000000004',
 'Tamora Group - 1.0000000000000004',
 'Eureka - 1.0000000000000004',
 'PT Mignon Sista Internasional - 1.0000000000000004',
 'Scremous - 1.0000000000000004',
 'PT Vinco - 1.0000000000000004',
 'Garuda Totalindo - 0.939354063262528',
 'PING Textile - 0.939354063262528',
 'Meccaya - 0.939354063262528',
 'Luby Indonesia - 0.939354063262528',
 'Airindo and Kobelindo - 0.939354063262528',
 'Delwin - 0.939354063262528',
 'PT Ernest Agribisnis - 0.939354063262528',
 'Telering - 0.939354063262528',
 'PT Kota Minyak Internasional - 0.939354063262528']